# Import the libraries and features

In [24]:
import numpy as np

features = np.load('features.npy')
labels = np.load('labels.npy')

print(features.shape)
print(labels.shape)

(199991, 20)
(200000,)


# Todo

implement a linear classifier on our features